# Correctness Test

## Set up

In [ ]:
import numpy as np
import time
from neo4jConnector import startNeo4jSession

In [ ]:
driver, session = startNeo4jSession(port=7688)

In [ ]:
## parameter settings
start_block_height = 164467
end_block_height = start_block_height + 1


# original set
b = 'Block'
t = 'Transaction'
a = 'Address'

# test set
bt = b + '_test'
tt = t + '_test'
at = a + '_test'

# analysis parameters
blockClasses       = [b, bt]
transactionClasses = [t, tt]
addressClasses     = [a, at]

## Analysis

### Functions

#### Queries

In [ ]:
### Blocks
## properties
def getBlockPropertiesByHeightQuery(blockClasses, height):
    return f'''
            MATCH (b:{blockClasses[0]} {{height: {height}}}) 
            WITH COLLECT([b.height, b.blockDate, b.hash, b.mediantime]) as properties
            RETURN apoc.util.md5(properties), properties
            UNION
            MATCH (b:{blockClasses[1]} {{height: {height}}}) 
            WITH COLLECT([b.height, b.blockDate, b.hash, b.mediantime]) as properties
            RETURN apoc.util.md5(properties), properties
            '''

###  Transactions
## retrieve transaction ids from block
def getTransactionsFromBlockHeightQuery(blockClasses, height):
    return f'''
            MATCH (b:{blockClasses[0]} {{height: {height}}})<-[:BELONGS_TO]-(t) 
            WITH t.txid as txids WITH txids ORDER BY txids WITH COLLECT(txids) as list 
            RETURN apoc.util.md5(COLLECT(list)), list
            UNION
            MATCH (b:{blockClasses[1]} {{height: {height}}})<-[:BELONGS_TO]-(t) 
            WITH t.txid as txids WITH txids ORDER BY txids WITH COLLECT(txids) as list 
            RETURN apoc.util.md5(COLLECT(list)), list
            '''

## TR properties
def getTransactionPropertiesByTxidQuery(transactionClasses, txid):
    return f'''
            MATCH (t:{transactionClasses[0]} {{txid: \"{txid}\"}}) 
                WITH COLLECT([t.txid, t.inDegree, t.outDegree, t.inSum, t.outSum, t.date]) as properties 
                RETURN apoc.util.md5(properties), properties
            UNION
            MATCH (t:{transactionClasses[1]} {{txid: \"{txid}\"}}) 
                WITH COLLECT([t.txid, t.inDegree, t.outDegree, t.inSum, t.outSum, t.date]) as properties 
                RETURN apoc.util.md5(properties), properties
            '''                                 


# TR edges with edgetype, value & address
# in case of need to also separately retrieve list of addresses
'''
MATCH (t:{transactionClasses[1]} {{txid: \"{txid}\"}})-[r:SENDS|RECEIVES]-(a) 
    WITH type(r) AS type, r.value AS value, a.address AS address
    WITH type, value, address ORDER BY type, value, address
    WITH COLLECT([type, value, address]) AS edges, COLLECT(address) AS addresses
    RETURN apoc.util.md5(COLLECT(edges)), edges, addresses
'''
#
def getAddressFromTransactionIDQuery(txClass, txid):
    return f'''
            MATCH (t:{transactionClasses[0]} {{txid: \"{txid}\"}})-[r:SENDS|RECEIVES]-(a) 
                WITH type(r) AS type, r.value AS value, a.address AS address
                WITH type, value, address ORDER BY type, value, address
                WITH COLLECT([type, value, address]) AS edges
                RETURN apoc.util.md5(COLLECT(edges)), edges
            UNION
            MATCH (t:{transactionClasses[1]} {{txid: \"{txid}\"}})-[r:SENDS|RECEIVES]-(a) 
                WITH type(r) AS type, r.value AS value, a.address AS address
                WITH type, value, address ORDER BY type, value, address
                WITH COLLECT([type, value, address]) AS edges
                RETURN apoc.util.md5(COLLECT(edges)), edges
            '''



#### Other functions

In [ ]:
#############################################
############## other functions ##############

def runQuery(query):
    return session.run(query).values()


## second version 
def matchProperties(nodeClassList, 
                    query,
                    node_identifier, 
                    match_count, 
                    mismatches):
    
    #send query to neo4j
    result = runQuery(query(nodeClassList, node_identifier))

    # match
    if len(result) == 1:
        match_count += 1
    
    # mismatch
    elif len(result) == 2:
         # todo: extend outputs to dictionary containing all relevant characters
        outputs                              = {}
        outputs['node_type']                 = nodeClassList[0]
        outputs['node_identifier']           = node_identifier
        outputs['query']                     = query.__name__
        outputs['cyperQuery']                = query(nodeClassList, node_identifier)
        outputs[nodeClassList[0]+'_outputs'] = result[0][-1]
        outputs[nodeClassList[1]+'_outputs'] = result[1][-1]
        
        mismatches.append(outputs)
    
    # something went wrong
    else:
        raise Exception('Length of query output does not match.')

    # return match_count, mismatches and all results without the hash
    return match_count, mismatches, result[0][-1]


### Correctnes check

In [ ]:
# overall parameter
mismatches  = []
match_count = 0 

#timing
startTime = time.time()

#loop through blocks
for block_height in np.arange(start_block_height, end_block_height):
    
    # check whether returned block properties match
    match_count, mismatches, outputs = matchProperties(nodeClassList = blockClasses, 
                                                       query = getBlockPropertiesByHeightQuery,
                                                       node_identifier = block_height,
                                                       match_count = match_count,
                                                       mismatches = mismatches)
    
    # check whether returned txids  match
    match_count, mismatches, outputs = matchProperties(nodeClassList = blockClasses, 
                                                       query = getTransactionsFromBlockHeightQuery,
                                                       node_identifier = block_height,
                                                       match_count = match_count,
                                                       mismatches = mismatches)

    # loop through all provided txids of the correct block
    for txid in outputs:

        # match tx properties
        match_count, mismatches, outputs = matchProperties(nodeClassList = transactionClasses, 
                                                           query = getTransactionPropertiesByTxidQuery,
                                                           node_identifier = txid, 
                                                           match_count = match_count,
                                                           mismatches = mismatches)
        
        match_count, mismatches, outputs = matchProperties(nodeClassList = transactionClasses, 
                                                           query = getAddressFromTransactionIDQuery,
                                                           node_identifier = txid, 
                                                           match_count = match_count,
                                                           mismatches = mismatches)
        
        
endTime = np.round((time.time() - startTime),2)
print(f'Retrieval execution time in {endTime} seconds.')
print('Retrieved blocks: ', 1 + block_height - start_block_height)
print(f'\nMatches: {match_count}; Mismatches: {len(mismatches)}')



## Mismatch analysis

In [ ]:
count = 1
for mismatch in mismatches:
    print(f'\n*********************************************************************** \
            \nMismatch nr. {count} \
            \n*********************************************************************** ')
    for item in mismatch:
        print('-', item)
        print('  ', mismatch[item])
        print('----------')
    count += 1 

## Error analysis

In [ ]:
# automatically check for error reason / specify what is wrong, based on function
# iterate through mismatches

# Test section

In [ ]:
##  test settings
height=1
txid = '60a20bd93aa49ab4b28d514ec10b06e1829ce6818ec06cd3aabd013ebcdc4bb1'
blockClass = b
txClass = t
addrClass = a

In [ ]:
txid = '60a20bd93aa49ab4b28d514ec10b06e1829ce6818ec06cd3aabd013ebcdc4bb1' #there
txid = 'd88fbb53146b8662b9e3d58f245d5d7ae0a841e8eb1ad13e8e3e8504a0d8fcea' #missing
txid = '9205ce5bdf62cdf04050b302150f5bcf62d998abece5745ade3b996c4889074e' #outsum previously not matching
txid = '60a20bd93aa49ab4b28d514ec10b06e1829ce6818ec06cd3aabd013ebcdc4bb1' #multisig one
#query = getTransactionPropertiesByTxidQuery(transactionClasses, txid)

height = 728
#height = 164467

#query = getTransactionsFromBlockHeightQuery(blockClasses, height)

query = getBlockPropertiesByHeightQuery(blockClasses, height)

#query = getAddressFromTransactionIDQuery(transactionClasses, txid)

print(query)
r = session.run(query).values()
print('length: ', len(r))
r

## Deleting all testnodes

In [ ]:
# delete all test nodes
delete_queries = [
    'match (t:Block_test) detach delete t',
    'match (t:Address_test) detach delete t',
    'match (t:Transaction_test) detach delete t']

for query in delete_queries:
    session.run(query)

## End connection

In [ ]:
session.close()
driver.close()